In [1]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import pandas as pd

all_reviews = 0
url = "https://letterboxd.com/film/beijing-watermelon/"
url = url + "/reviews/by/activity/page/"
keyword = "cinema"

async def fetch_page(session, url):
    async with session.get(url) as response:
        return await response.text()

async def parse_page(url, session):
    html = await fetch_page(session, url)
    soup = BeautifulSoup(html, "html.parser")
    return soup

async def collect_data(review_item, keyword):
    review_text = review_item.find('div', class_="body-text -prose collapsible-text").text
    if review_text.endswith('...'):
        review_text = await parse_page(full_review_url, session)
    
    relevance = keyword in review_text

    if relevance:
        full_review_url = "https://letterboxd.com" + review_item.find('div', class_="body-text -prose collapsible-text")['data-full-text-url']

        try:
            rating = review_item.find('span', class_=lambda value: value and 'rated' in value).text
        except:
            rating = None

        return [full_review_url, rating, review_text]

    return None

async def get_reviews_page(page_url, keyword, session):
    page = await parse_page(page_url, session)
    all_reviews = page.find_all('div', {'class': 'film-detail-content'})
    review_data = [collect_data(review, keyword) for review in all_reviews]
    return [data for data in review_data if data is not None]

async def get_reviews(url, keyword):
    all_review_data = pd.DataFrame(columns=['URL', 'Rating', 'Review'])
    async with aiohttp.ClientSession() as session:
        for i in range(1, 100):  # Adjust the range accordingly
            page_url = url + str(i)
            reviews = await get_reviews_page(page_url, keyword, session)
            if not reviews:
                break

            df_row = pd.DataFrame(reviews, columns=['URL', 'Rating', 'Review'])
            all_review_data = pd.concat([all_review_data, df_row])

    return all_review_data

async def main():
    data = await get_reviews(url=url, keyword=keyword)
    print(data)

if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop